In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install rouge_score
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.2 MB/s 
     |████████████████████████████████| 163 kB 68.5 MB/s 
     |████████████████████████████████| 6.6 MB 85.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 2.1 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 115 kB 61.1 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c4d76a1c3687cc0857817da63629a33aadb6650e0bc1f

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("eli5")
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


  0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b"]:
  qprefix = "question:"
  cprefix = "context:"
else:
  qprefix = ""
  cprefix = ""

In [ ]:
inputs = [qprefix + " " + doc[0] + " " + cprefix + " " + doc[1] for doc in zip(raw_datasets['train_eli5'][:2]["title"], raw_datasets['train_eli5'][:2]['selftext'])]

In [ ]:
max_input_length = 128
max_target_length = 512

def preprocess_data(examples):
  inputs = [qprefix + " " + doc[0] + " " + cprefix + " " + doc[1] for doc in zip(examples["title"], examples['selftext'])]
  # print(inputs)
  model_inputs = tokenizer(inputs, max_length = max_input_length, truncation = True)

  with tokenizer.as_target_tokenizer():
    texts = [x['text'][0] for x in examples['answers']]
    labels = tokenizer([" ".join(p['text']) for p in examples['answers']], max_length = max_target_length, truncation = True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
preprocess_data(raw_datasets['train_eli5'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[822, 10, 16, 3370, 125, 7, 8, 500, 13, 3, 26281, 8, 166, 192, 4805, 28, 3, 9, 10505, 3, 18, 95, 8, 2214, 3, 18, 59, 1646, 10505, 4805, 3, 23, 129, 273, 2625, 10, 1], [822, 10, 1615, 33, 315, 3, 3276, 7, 41, 60, 122, 4885, 3, 2, 2076, 3, 2, 3331, 61, 13, 1807, 31, 1596, 966, 373, 335, 3151, 7, 315, 58, 2625, 10, 27, 31, 162, 4944, 24, 8, 1750, 16, 594, 344, 1646, 1807, 11, 2076, 5517, 6, 11, 344, 2076, 5517, 11, 3331, 6, 19, 966, 373, 335, 3151, 7, 5, 100, 1330, 12, 1520, 1176, 150, 1052, 125, 8, 594, 21, 1646, 1807, 5, 100, 744, 31, 17, 1727, 12, 143, 1254, 6, 38, 8, 1750, 344, 3771, 11, 3771, 5, 1714, 11, 8, 1750, 344, 7514, 11, 7514, 5, 1714, 3, 87, 6191, 33, 7385, 1427, 182, 315, 5, 27, 7, 48, 131, 46, 3, 26968, 8346, 24, 23248, 7, 12874, 13, 3, 9, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
tokenized_dataset = raw_datasets.map(preprocess_data, batched = True)

  0%|          | 0/273 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/132 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/99 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-eli5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train_eli5"],
    eval_dataset=tokenized_dataset["validation_eli5"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5 into local empty directory.


Download file pytorch_model.bin:   0%|          | 582/231M [00:00<?, ?B/s]

Download file runs/Sep26_03-36-13_87070f92ad8d/events.out.tfevents.1664174163.87070f92ad8d.325.4: 100%|#######…

Download file training_args.bin: 100%|##########| 3.42k/3.42k [00:00<?, ?B/s]

Download file runs/Sep26_03-24-44_87070f92ad8d/1664163293.894326/events.out.tfevents.1664163293.87070f92ad8d.3…

...

Clean file runs/Sep26_03-36-13_87070f92ad8d/events.out.tfevents.1664163386.87070f92ad8d.325.2:   9%|9         …

Clean file runs/Sep26_03-36-13_87070f92ad8d/1664163386.3543332/events.out.tfevents.1664163386.87070f92ad8d.325…

Clean file runs/Sep26_03-24-44_87070f92ad8d/events.out.tfevents.1664163253.87070f92ad8d.325.0:  20%|##        …

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: selftext, q_id, answers, subreddit, document, title, title_urls, selftext_urls, answers_urls. If selftext, q_id, answers, subreddit, document, title, title_urls, selftext_urls, answers_urls are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 272634
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 51120


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.980600,3.772620,9.847500,1.872000,7.946200,9.125800,18.997200
2,3.945800,3.736945,9.923200,1.898100,7.992200,9.206100,18.998800
3,3.935500,3.727522,9.944000,1.908000,8.014500,9.227500,18.998800


Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-500
Configuration saved in t5-small-finetuned-eli5/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-finetuned-eli5/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-1000
Configuration saved in t5-small-finetuned-eli5/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5-sma

Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-17500
Configuration saved in t5-small-finetuned-eli5/checkpoint-17500/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-17500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-17500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-17500/special_tokens_map.json
Deleting older checkpoint [t5-small-finetuned-eli5/checkpoint-16000] due to args.save_total_limit
Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-18000
Configuration saved in t5-small-finetuned-eli5/checkpoint-18000/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-18000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-18000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-18000/special_tokens_map.json
Deleting older checkpoint [t5-small-finetuned-eli5/checkpoint-16500] due t

TrainOutput(global_step=51120, training_loss=3.984574078244969, metrics={'train_runtime': 26384.9923, 'train_samples_per_second': 30.999, 'train_steps_per_second': 1.937, 'total_flos': 2.5520093437231104e+16, 'train_loss': 3.984574078244969, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-finetuned-eli5
Configuration saved in t5-small-finetuned-eli5/config.json
Model weights saved in t5-small-finetuned-eli5/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/231M [00:00<?, ?B/s]

Upload file runs/Oct01_18-00-00_4f09b0f4bcfd/events.out.tfevents.1664647576.4f09b0f4bcfd.74.0:  15%|#4        …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   296728c..0ce2611  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   296728c..0ce2611  main -> main

To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   0ce2611..1193370  main -> main

   0ce2611..1193370  main -> main



'https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5/commit/0ce2611848f660345801cba40b67e93ab11dd94a'

In [ ]:
metrics = trainer.evaluate(tokenized_dataset['validation_eli5'])

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: selftext, q_id, answers, subreddit, document, title, title_urls, selftext_urls, answers_urls. If selftext, q_id, answers, subreddit, document, title, title_urls, selftext_urls, answers_urls are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9812
  Batch size = 16
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
#### Test Metrics ######

metrics 

{'eval_loss': 9.68708324432373,
 'eval_rouge1': 3.3563,
 'eval_rouge2': 0.6013,
 'eval_rougeL': 2.8018,
 'eval_rougeLsum': 3.1173,
 'eval_gen_len': 7.8989,
 'eval_runtime': 261.91,
 'eval_samples_per_second': 37.463,
 'eval_steps_per_second': 2.344}